In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [2]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
ffK(s) = (@sprintf "%.5g" sd(3,s)/1e3)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe8(s) = (@sprintf "%.4g" sd(3,s)*1e8)

ffe8 (generic function with 1 method)

In [3]:
@load "icc_test_small.jl.jld2"
dic_icc = copy(dic)

Dict{Any,Any} with 9 entries:
  "names"     => ["icc"]
  "icc_solve" => [0.194866, 0.18172, 0.200608, 0.220001, 0.19937, 0.32742, 0.19…
  "icc_its"   => [21.0, 22.0, 33.0, 65.0, 52.0, 232.0, 40.0, 62.0, 27.0, 46.0  …
  "ne"        => [53364, 90000, 35334, 60284, 90000, 46148, 30828, 41614, 96292…
  "testName"  => ["uni_chimera(10000,1)", "uni_chimera(10000,2)", "uni_chimera(…
  "icc_build" => [0.016394, 0.017483, 0.016748, 0.016273, 0.018803, 0.016873, 0…
  "icc_err"   => [8.6371e-9, 5.18389e-9, 8.69277e-9, 8.93221e-9, 6.58878e-9, 9.…
  "nv"        => [10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000…
  "icc_tot"   => [0.21126, 0.199203, 0.217356, 0.236274, 0.218173, 0.344293, 0.…

In [4]:
uni_chimera_indices = findall(entry -> contains(entry, "uni_chimera(10000,"), dic_icc["testName"])
wted_chimera_indices = findall(entry -> contains(entry, "wted_chimera(10000,"), dic_icc["testName"])
uni_bndry_chimera_indices = findall(entry -> contains(entry, "uni_bndry_chimera(10000,"), dic_icc["testName"])
wted_bndry_chimera_indices = findall(entry -> contains(entry, "wted_bndry_chimera(10000,"), dic_icc["testName"])
suitesparse_indices = setdiff(1:length(dic_icc["testName"]), vcat(uni_chimera_indices, wted_chimera_indices, uni_bndry_chimera_indices, wted_bndry_chimera_indices));


In [5]:
problem_idx = [suitesparse_indices, uni_chimera_indices, wted_chimera_indices, uni_bndry_chimera_indices, wted_bndry_chimera_indices]
problem_names = ["suitesparse", "uni_chimera", "wted_chimera", "uni_bndry_chimera", "wted_bndry_chimera"]

5-element Array{String,1}:
 "suitesparse"
 "uni_chimera"
 "wted_chimera"
 "uni_bndry_chimera"
 "wted_bndry_chimera"

In [6]:
ratios = []
errors = []
for indices in problem_idx
    ratio = dic_icc["icc_tot"][indices] ./ dic_icc["ne"][indices]
    push!(ratios, ratio)
    error = dic_icc["icc_err"][indices]
    push!(errors, error)
end


In [7]:
for (name, ratio, error) in zip(problem_names, ratios, errors)
    mean_val = mean(ratio)
    quantile_val = quantile(ratio, 0.75)
    max_val = maximum(ratio)
    
    @printf("%-20s - Ratio - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n", name, ffmicro(mean_val), ffmicro(quantile_val), ffmicro(max_val))
    mean_error = mean(error)
    quantile_error = quantile(error, 0.75)
    max_error = maximum(error)
    
    @printf("\u001b[31m%-20s - Error - Mean: %10s | 0.75 Quantile: %10s | Max: %10s\n\u001b[0m", name, ffe8(mean_error), ffe8(quantile_error), ffe8(max_error))
end

suitesparse          - Ratio - Mean:       29.8 | 0.75 Quantile:       42.2 | Max:        147
suitesparse          - Error - Mean:      0.366 | 0.75 Quantile:      0.739 | Max:      0.996
uni_chimera          - Ratio - Mean:       4.22 | 0.75 Quantile:       5.17 | Max:       11.3
uni_chimera          - Error - Mean:   2.91e+06 | 0.75 Quantile:      0.919 | Max:      1e+08
wted_chimera         - Ratio - Mean:       4.81 | 0.75 Quantile:        5.5 | Max:       14.3
wted_chimera         - Error - Mean:   2.91e+06 | 0.75 Quantile:      0.933 | Max:      1e+08
uni_bndry_chimera    - Ratio - Mean:       4.26 | 0.75 Quantile:       5.35 | Max:       7.33
uni_bndry_chimera    - Error - Mean:      0.682 | 0.75 Quantile:       0.82 | Max:      0.989
wted_bndry_chimera   - Ratio - Mean:       4.58 | 0.75 Quantile:       5.61 | Max:       10.2
wted_bndry_chimera   - Error - Mean:      0.708 | 0.75 Quantile:      0.893 | Max:      0.993


In [14]:
argmax_ratio = problem_idx[2][argmax(ratios[2])]
dic_icc["testName"][argmax_ratio]


"uni_chimera(10000,59)"

In [22]:
a = uni_chimera(10000,59)
b = randn(10000)
b = (b .- mean(b))
b = b./ norm(b);

In [18]:
include("../julia-files/compareSolvers.jl")
include("../julia-files/matlabSafe.jl")

timeLimitRchol (generic function with 1 method)

In [24]:
timeLimitIcc(Inf, lap(a), b; verbose=true)

MATLAB is selecting SOFTWARE OPENGL rendering.

                                                                              < M A T L A B (R) >
                                                                    Copyright 1984-2020 The MathWorks, Inc.
                                                                R2020b Update 3 (9.9.0.1538559) 64-bit (glnxa64)
                                                                               November 23, 2020

 
For online documentation, see https://www.mathworks.com/support
For product information, visit www.mathworks.com.
 
>> >> >> >> >> >> >> >> >> >> >> 
n =

       10000

>> >> >> >> >> >> >> >> >> >> >> >> >> 
relres =

   8.7245e-09

>> >> >> >> >> Build Time: 0.157044
Solve Time: 0.482753
Iterations: 413
error: 8.724502358764304e-9
Time to load and start Matlab: 6866 milliseconds/1000)


(0.482753, 0.157044, 413, 8.724502358764304e-9)

In [26]:
ffmicro((0.157044 + 0.482753) / nnz(lap(a)))

"20.7"

In [27]:
argmax_err = problem_idx[2][argmax(errors[2])]
dic_icc["testName"][argmax_err]

"uni_chimera(10000,25)"

In [28]:
a = uni_chimera(10000,25)
b = randn(10000)
b = (b .- mean(b))
b = b./ norm(b);

In [29]:
timeLimitIcc(Inf, lap(a), b; verbose=true)

MATLAB is selecting SOFTWARE OPENGL rendering.

                                                                              < M A T L A B (R) >
                                                                    Copyright 1984-2020 The MathWorks, Inc.
                                                                R2020b Update 3 (9.9.0.1538559) 64-bit (glnxa64)
                                                                               November 23, 2020

 
For online documentation, see https://www.mathworks.com/support
For product information, visit www.mathworks.com.
 
>> >> >> >> >> >> >> >> >> >> >> 
n =

       10000

>> >> >> >> >> >> >> >> 

Error using ichol
Encountered nonpositive pivot.
 


>> >> >> >> >> 

Unrecognized function or variable 'L'.
 
Unrecognized function or variable 'xs'.
 



relres =

     1

>> >> >> >> >> Build Time: 0.033863
Solve Time: 0.102437
Iterations: Inf
error: 1.0
Time to load and start Matlab: 7458 milliseconds/1000)


(0.102437, 0.033863, Inf, 1.0)